# Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

import urllib.request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

try:
    import folium
except:
    !pip install folium
    import folium
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# Scraping data from Wikipedia

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")
all_tables

right_table=soup.find('table', class_='wikitable sortable')
right_table

postalcode = []
borough = []
neighbourhood = []
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postalcode.append(cells[0].find(text = True))
        borough.append(cells[1].find(text = True))
        neighbourhood.append(cells[2].find(text = True))

df=pd.DataFrame(postalcode,columns=['Postal Code'])
df['Borough']=borough
df['Neighbourhood']=neighbourhood
df.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Dropping 'Not assigned' rows of Borough

In [4]:
df = df[~df.Borough.str.contains("Not assigned")].reset_index(drop = True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Grouping the data

In [5]:
df = df.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


# For 'Not assigned' Neighbourhood assign the same value as Borough

In [6]:
for index, row in df.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


# Number of rows in Dataframe

In [7]:
df.shape

(103, 3)

# Loading the coordinates from csv file

In [8]:
coord = pd.read_csv("http://cocl.us/Geospatial_data")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Concatinating the dataframes 

In [13]:
coord_only = coord[['Latitude', 'Longitude']]
new_df = pd.concat([df, coord_only], axis = 1)
print(" ",new_df.shape)
new_df.head()

  (103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge\n",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


# Using geopy library to get Latitude and Longitude of Toronto

In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


# Creating map of Toronto with neighbourhoods superimposed on top

In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Filtering Borough that contains Toronto

In [15]:
borough_names = list(new_df.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto\n', 'Central Toronto\n', 'Downtown Toronto\n', 'West Toronto\n']

In [16]:
new_df = new_df[new_df['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(new_df.shape)
new_df.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches\n,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale\n",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West\n",43.668999,-79.315572
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park\n,43.728020,-79.388790


# Using Foursquare API to explore neighbourhoods

In [17]:
CLIENT_ID = 'SWPLTFHWWVJMJLWTUJDCDHDTHSOWFK0S5YLOMJWWXHNEJDPY' 
CLIENT_SECRET = 'WVZO04SI4NJLYVF5WJVNVWGPMV3TO5LZ1OW5ZO1DIKUCUOWU' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SWPLTFHWWVJMJLWTUJDCDHDTHSOWFK0S5YLOMJWWXHNEJDPY
CLIENT_SECRET:WVZO04SI4NJLYVF5WJVNVWGPMV3TO5LZ1OW5ZO1DIKUCUOWU


# Getting top 100 venues within a radius of 500

In [20]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Postal Code'], new_df['Borough'], new_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
print('done')

done


In [34]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1640, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches\n,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches\n,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches\n,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches\n,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale\n",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


# Getting number of venues for each POstal Code

In [35]:
venues_df.groupby(["Postal Code", "Borough", "Neighborhood"]).count()


,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postal Code,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches\n,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale\n",41,41,41,41,41,41
M4L,East Toronto,"India Bazaar, The Beaches West\n",20,20,20,20,20,20
M4M,East Toronto,Studio District\n,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park\n,4,4,4,4,4,4
M4P,Central Toronto,Davisville North\n,10,10,10,10,10,10
M4R,Central Toronto,"North Toronto West, Lawrence Park\n",17,17,17,17,17,17
M4S,Central Toronto,Davisville\n,34,34,34,34,34,34
M4T,Central Toronto,"Moore Park, Summerhill East\n",4,4,4,4,4,4


# Analyzing each area

In [36]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_onehot['Postal Code'] = venues_df['Postal Code'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1640, 241)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches\n,0,0,0,0,0,0,0,0,0

# Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby(["Postal Code", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 241)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches\n,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0

# Getting top 10 venues for each Postal Code

In [41]:
 num_top_venues = 10
indicators = ['st', 'nd', 'rd']

areaColumns = ['Postal Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,Postal Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches\n,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale\n",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Liquor Store,Indian Restaurant,Spa,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West\n",Sandwich Place,Pizza Place,Sushi Restaurant,Pub,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Steakhouse,Fish & Chips Shop
3,M4M,East Toronto,Studio District\n,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park\n,Park,Bus Line,Swim School,Dim Sum Restaurant,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M4P,Central Toronto,Davisville North\n,Gym / Fitness Center,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Dance Studio,Park,Pizza Place,Sandwich Place,Gym
6,M4R,Central Toronto,"North Toronto West, Lawrence Park\n",Clothing Store,Coffee Shop,Yoga Studio,Furniture / Home Store,Chinese Restaurant,Café,Restaurant,Salon / Barbershop,Diner,Mexican Restaurant
7,M4S,Central Toronto,Davisville\n,Sandwich Place,Dessert Shop,Pizza Place,Café,Coffee Shop,Gym,Italian Restaurant,Sushi Restaurant,Diner,Indian Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East\n",Restaurant,Playground,Summer Camp,Trail,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Sushi Restaurant,Fried Chicken Joint,Liquor Store,Sandwich Place,Restaurant,Sports Bar,Bank,Bagel Shop


# Clustering areas

In [42]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Postal Code", "Borough", "Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [43]:
toronto_merged = new_df.copy()

toronto_merged["Cluster Labels"] = kmeans.labels_

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("Postal Code"), on="Postal Code")

print(toronto_merged.shape)
toronto_merged.head() 

(39, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches\n,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale\n",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Liquor Store,Indian Restaurant,Spa,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West\n",43.668999,-79.315572,1,Sandwich Place,Pizza Place,Sushi Restaurant,Pub,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Steakhouse,Fish & Chips Shop
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park\n,43.728020,-79.388790,1,Park,Bus Line,Swim School,Dim Sum Restaurant,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# Sorting the results by Cluster Labels

In [44]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches\n,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange\n",43.647177,-79.381576,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Salad Place,American Restaurant,Steakhouse,Sporting Goods Shop
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel\n",43.648198,-79.379817,1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Seafood Restaurant,Deli / Bodega,Beer Bar,Bar
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville\n",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,Pub,History Museum,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,BBQ Joint,Liquor Store
25,M5S,Downtown Toronto,"University of Toronto, Harbord\n",43.662696,-79.400049,1,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Dessert Shop,Pub,Chinese Restaurant
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park\n",43.653206,-79.400049,1,Coffee Shop,Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Park,Dessert Shop,Bakery,Caribbean Restaurant,Pizza Place
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Boutique,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport Terminal
28,M5W,Downtown Toronto,Stn A PO Boxes\n,43.646435,-79.374846,1,Coffee Shop,Café,Pub,Hotel,Restaurant,Seafood Restaurant,Beer Bar,Gym,Japanese Restaurant,Italian Restaurant
29,M5X,Downtown Toronto,"First Canadian Place, Underground city\n",43.648429,-79.382280,1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Steakhouse,American Restaurant,Seafood Restaurant,Asian Restaurant
30,M6G,Downtown Toronto,Christie\n,43.669542,-79.422564,1,Grocery Store,Café,Park,Candy Store,Nightclub,Baby Store,Athletics & Sports,Italian Restaurant,Diner,Restaurant


# Visualizing the resulting clusters

In [48]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining the clusters

## Cluster 1

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


## Cluster 2

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Salad Place,American Restaurant,Steakhouse,Sporting Goods Shop
21,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Seafood Restaurant,Deli / Bodega,Beer Bar,Bar
24,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Pub,History Museum,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,BBQ Joint,Liquor Store
25,Downtown Toronto,1,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Dessert Shop,Pub,Chinese Restaurant
26,Downtown Toronto,1,Coffee Shop,Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Park,Dessert Shop,Bakery,Caribbean Restaurant,Pizza Place
27,Downtown Toronto,1,Airport Service,Airport Lounge,Boutique,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport Terminal
28,Downtown Toronto,1,Coffee Shop,Café,Pub,Hotel,Restaurant,Seafood Restaurant,Beer Bar,Gym,Japanese Restaurant,Italian Restaurant
29,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Steakhouse,American Restaurant,Seafood Restaurant,Asian Restaurant
30,Downtown Toronto,1,Grocery Store,Café,Park,Candy Store,Nightclub,Baby Store,Athletics & Sports,Italian Restaurant,Diner,Restaurant
31,West Toronto,1,Bakery,Pharmacy,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Café,Brewery,Bar,Bank


## Cluster 3

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Jewelry Store,Park,Sushi Restaurant,Trail,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


## Cluster 4

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


## Cluster 5

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Home Service,Garden,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


# Conclusion

###### Most of the neighbourhoods fall into cluster 2, with mostly budiness area like cafe, restaurant, hotel etc 